In [ ]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset of 50K Movie Reviews/IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [ ]:
import nltk
nltk.download('stopwords')
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def load_dataset():
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset of 50K Movie Reviews/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
45164    [a, huge, cast, gathered, remake, sadly, box, ...
41022    [jim, varney, performances, harem, girl, a, u,...
19021    [oh, come, people, give, film, break, the, one...
22773    [ok, i, kid, typical, gay, love, life, since, ...
8350     [i, wish, return, jedi, directed, somebody, el...
                               ...                        
31448    [like, terrible, cancer, raining, sky, i, wand...
36485    [if, derivative, predictable, rape, revenge, t...
30933    [if, enjoy, original, snl, cast, shows, avoid,...
40419    [one, dark, night, typical, teen, horror, film...
37234    [when, one, stops, recollect, upon, frequent, ...
Name: review, Length: 40000, dtype: object 

41622    [vertigo, co, stars, stewart, last, turn, roma...
44139    [i, love, anthony, hopkins, actor, i, interest...
31265    [jim, mckay, made, one, best, films, see, year...
11092    [i, almost, tempted, demand, money, back, vide...
3313     [i, really, like, traci, lords, she, may, grea...
 

In [ ]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [ ]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[   39   550    86 ...     0     0     0]
 [ 1125 13762   271 ...     0     0     0]
 [  325   122    20 ... 56961  1142  1083]
 ...
 [   55   265   119 ...     0     0     0]
 [    5   360   212 ...   232    81    27]
 [  171     5  2943 ...  3930  1831  5230]] 

Encoded X Test
 [[ 8110   948   307 ...     0     0     0]
 [    1    40  2058 ...     0     0     0]
 [ 1125 18741    24 ...     0     0     0]
 ...
 [    8     3    36 ...     0     0     0]
 [    1  8252   743 ...     0     0     0]
 [  217   256     1 ...     0     0     0]] 

Maximum review length:  130


In [ ]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2957856   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,982,753
Trainable params: 2,982,753
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [ ]:
import time

start_time = time.time()

model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

end_time = time.time()
total_time = end_time - start_time
print("Total training time:", total_time, "seconds")

Epoch 1/5
313/313 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.4984
Epoch 1: accuracy improved from -inf to 0.49843, saving model to models/LSTM.h5
313/313 [==============================] - 89s 273ms/step - loss: 0.6932 - accuracy: 0.4984
Epoch 2/5
313/313 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.4963
Epoch 2: accuracy did not improve from 0.49843
313/313 [==============================] - 85s 272ms/step - loss: 0.6932 - accuracy: 0.4963
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.4940
Epoch 3: accuracy did not improve from 0.49843
313/313 [==============================] - 96s 307ms/step - loss: 0.6932 - accuracy: 0.4940
Epoch 4/5
313/313 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.4999
Epoch 4: accuracy improved from 0.49843 to 0.49988, saving model to models/LSTM.h5
313/313 [==============================] - 86s 274ms/step - loss: 0.6932 - accuracy: 0.

In [ ]:
# Get predicted probabilities from the model
y_pred = model.predict(x_test, batch_size=128)

# Set a custom threshold
threshold = 0.3

# Convert probabilities to class labels based on threshold
y_pred_classes = (y_pred > threshold).astype(int)

# Calculate accuracy
true = 0
for i, y in enumerate(y_test):
    if y == y_pred_classes[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred_classes) - true))
print('Accuracy: {}'.format(true/len(y_pred_classes)*100))


79/79 [==============================] - 8s 89ms/step
Correct Prediction: 4973
Wrong Prediction: 5027
Accuracy: 49.730000000000004


In [ ]:
loaded_model = load_model('models/LSTM.h5')

In [ ]:
review = str(input('Movie Review: '))

Movie Review: Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!


In [ ]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  Nothing was typical about this Everything was beautifully done in this movie the story the flow the scenario everything I highly recommend it for mystery lovers for anyone who wants to watch a good movie
Filtered:  ['nothing typical everything beautifully done movie story flow scenario everything i highly recommend mystery lovers anyone wants watch good movie']


In [ ]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  78  707  174 1179  127    3   13 2703 2526  174    1  442  282  683
  1771  152  395   33    9    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


In [ ]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 [==============================] - 1s 794ms/step
[[0.5006674]]


In [ ]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

negative
